# 주식 및 ETF 데이터 수집

In [2]:
# !pip install pykrx
# !pip install beautifulsoup4 pandas openpyxl selenium
# !pip install webdriver-manager

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pykrx import stock
import datetime
import os


In [5]:

# 저장된 파일이 있는 폴더 경로
folder_path = '.'
# 조건 설정
per_threshold = 10
pbr_threshold = 1
div_threshold = 3


# 현재 날짜 기준으로 최근 월 종목 리스트 수집
current_date = datetime.datetime.now()
last_month_str = current_date.strftime("%Y%m") + '01'

tickers_kosdaq = stock.get_market_ticker_list(last_month_str, market='KOSDAQ')
tickers_kospi = stock.get_market_ticker_list(last_month_str, market='KOSPI')
recent_tickers = set(tickers_kosdaq + tickers_kospi)

# 각 티커에 대한 처리
for ticker in recent_tickers:
    # 티커의 마지막 자리가 0이 아니면 건너뜀
    if ticker[-1] != '0':
        continue

    file_path = os.path.join(folder_path, f"{ticker}.xlsx")

    # 파일이 존재하지 않으면 새로 생성
    if not os.path.exists(file_path):
        df1 = stock.get_market_ohlcv("19800102", current_date.strftime("%Y%m%d"), ticker)
            # df1 또는 df2 중 하나라도 비어 있으면 다음 티커로 넘어감
        if df1.empty:
            continue
        df2 = stock.get_market_fundamental("19800102", current_date.strftime("%Y%m%d"), ticker)
        if df2.empty:
            continue
        df3 = stock.get_market_cap("19800102", current_date.strftime("%Y%m%d"), ticker)
        combined_df = pd.concat([df1, df2, df3], axis=1, join='outer')
                # '고가' 컬럼이 있는지 확인
        if '고가' not in combined_df.columns:
            continue
                # 조건 확인
        if ('PER' in combined_df.columns and 'PBR' in combined_df.columns and 'DIV' in combined_df.columns):
            condition = (combined_df['PER'] > 0) & (combined_df['PER'] <= per_threshold) & \
                        (combined_df['PBR'] > 0) & (combined_df['PBR'] <= pbr_threshold) & \
                        (combined_df['DIV'] >= div_threshold)
            if not condition.any():
                continue  # 조건을 만족하지 않으면 다음 티커로 넘어감
        else:
            continue  # 필요한 컬럼이 없으면 넘어감
        # 노멀라이즈드 값 계산
        combined_df['normalized_value'] = (combined_df['종가'] - combined_df['종가'].rolling(window=252*5, min_periods=252).min()) / \
                                          (combined_df['종가'].rolling(window=252*5, min_periods=252).max() - combined_df['종가'].rolling(window=252*5, min_periods=252).min()) * 100
        
        combined_df.reset_index(inplace=True)
        combined_df['ticker'] = ticker
        combined_df.rename(columns={
            '날짜': 'date',
            '시가': 'open',
            '고가': 'high',
            '저가': 'low',
            '종가': 'close',
            '거래량': 'volume',
            '거래대금': 'value',
            '시가총액': 'market_cap',
            '상장주식수': 'shares_outstanding'
        }, inplace=True)
        
        # 엑셀 파일로 저장
        combined_df.to_excel(file_path, index=False)
        print(f"새 파일 생성: {ticker}")
    else:
        # 기존 파일 로드 및 업데이트
        existing_df = pd.read_excel(file_path, index_col=0)
        # 마지막 날짜 확인 및 새로운 데이터 수집
        last_date = existing_df.index[-1]
        next_date = (pd.to_datetime(last_date) + datetime.timedelta(days=1)).strftime("%Y%m%d")
        # 기존 데이터에서 중복을 피하기 위해 next_date 이후 데이터 삭제
        existing_df = existing_df[:last_date]
        new_ohlcv = stock.get_market_ohlcv(next_date, current_date.strftime("%Y%m%d"), ticker)
        new_fundamental = stock.get_market_fundamental(next_date, current_date.strftime("%Y%m%d"), ticker)
        new_cap = stock.get_market_cap(next_date, current_date.strftime("%Y%m%d"), ticker)

        
        new_combined_df = pd.concat([new_ohlcv, new_fundamental, new_cap], axis=1, join='outer')
          # 열 이름과 순서 일치시키기 전에 열의 수를 확인
        if len(new_combined_df.columns) != len(existing_df.columns):
            # print(f"열의 수가 일치하지 않음: {ticker}")
            # print(new_combined_df.columns)
            continue
         # 노멀라이즈드 값 계산
        new_combined_df['normalized_value'] = (new_combined_df['종가'] - new_combined_df['종가'].rolling(window=252*5, min_periods=252).min()) / \
                                              (new_combined_df['종가'].rolling(window=252*5, min_periods=252).max() - new_combined_df['종가'].rolling(window=252*5, min_periods=252).min()) * 100

        new_combined_df.reset_index(inplace=True)
        new_combined_df['ticker'] = ticker
        new_combined_df.rename(columns={
            '날짜': 'date',
            '시가': 'open',
            '고가': 'high',
            '저가': 'low',
            '종가': 'close',
            '거래량': 'volume',
            '거래대금': 'value',
            '시가총액': 'market_cap',
            '상장주식수': 'shares_outstanding'
        }, inplace=True)
        
        # 기존 데이터와 새로운 데이터를 병합하여 업데이트
        updated_df = pd.concat([existing_df, new_combined_df]).drop_duplicates(subset=['date', 'ticker'])
        updated_df.to_excel(file_path, index=False)
        print(f"{ticker} 업데이트 완료")

새 파일 생성: 137310
새 파일 생성: 079430


KeyboardInterrupt: 

## ETF 티커 & 리스트 생성  

In [6]:
# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [8]:
# URL에서 HTML 컨텐츠를 가져옴
url = 'https://finance.naver.com/sise/etf.naver'
driver.get(url)

# 종목명과 티커를 추출하여 리스트로 저장
etf_data = []
rows = driver.find_elements(By.CSS_SELECTOR, '#etfItemTable > tr')
for row in rows:
    name_col = row.find_elements(By.CSS_SELECTOR, 'td.ctg a')
    if name_col:
        name = name_col[0].text.strip()
        ticker = name_col[0].get_attribute('href').split('=')[-1]  # 종목명 링크에서 티커 추출
        etf_data.append([name, ticker])

# 드라이버 종료
driver.quit()

# DataFrame 생성
df = pd.DataFrame(etf_data, columns=['종목명', '티커'])

# DataFrame을 Excel 파일로 저장
excel_filename = 'etf_list.xlsx'
df.to_excel(excel_filename, index=False)

print(f"Excel 파일로 저장되었습니다: {excel_filename}")


Excel 파일로 저장되었습니다: etf_list.xlsx
